In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import time

Strategy: 
- Code to collect movie title and other features into nested list
- Code to automate capture of data from multiple web pages and store in dataframe   

In [2]:
def get_info_v2(soup):
    
    # nested list of films. contains movie title and corresponding features
    movies_list = []
    
    specs = soup.find_all('div',{'class':'lister-item-content'})


    for content in specs:
        
        # contains the information identified in the variables - features of project
        movie_attributes = []

        movie_year = content.find('span',class_='lister-item-year text-muted unbold').text.replace("(", "").replace(")", "") 
        movie_title = content.find_all('a')[0].text

        # movie deets contains many features. Below we get them indiviudally 
        movie_deets = content.find_all('p')[0]
        movie_runtime = movie_deets.find('span', class_='runtime').text
        movie_genre = movie_deets.find('span', class_='genre').text.replace('\n','').strip()
        movie_rating = movie_deets.find('span', class_ = 'certificate').text
        director_stars = content.find_all('p')[2]
        

        # director and gross box office $
        #final_gross = float(votes_gross_split[:-2])
        director = director_stars.find('a').text
        blob = director_stars.findNext('a')
        actor1 = blob.findNext('a').text
        
        votes_gross = content.find('p', class_= "sort-num_votes-visible").text
        votes_gross_split = votes_gross.split('$')[1]
        final_gross = float(votes_gross_split[:-2])
        
        # places variables into movie attribute list
        movie_attributes.append(movie_title)
        movie_attributes.append(movie_runtime)
        movie_attributes.append(movie_genre)
        movie_attributes.append(movie_rating)
        movie_attributes.append(final_gross)
        movie_attributes.append(director)
        movie_attributes.append(actor1)
        movie_attributes.append(movie_year)
        
        # append movie attributes to list 
        movies_list.append(movie_attributes)

    return movies_list

In [3]:
start_pages = [751,801,851,901,951,1001,1051,1101]

#instantiates empty dataframe
master_df = pd.DataFrame()

master_list = []
for page in start_pages:
    all_url = 'https://www.imdb.com/search/title/?genres=thriller&sort=boxoffice_gross_us,desc&start=' + str(page) +'&explore=title_type,genres&ref_=adv_nxt'
    response = requests.get(all_url)
    page_html = response.text  
    soup = bs(page_html)
    curr_data = get_info_v2(soup)
    curr_movies = pd.DataFrame(curr_data, columns = ['Title', 'Duration', 'Genre','Rating', 'Gross','Director','Lead', 'Year']) 
    
    #concatonates each page into the empty dataframe
    master_df = pd.concat([master_df,curr_movies], axis=0)

In [4]:
master_df

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year
0,Dracula 2000,99 min,"Action, Fantasy, Horror",R,33.02,Patrick Lussier,Gerard Butler,2000
1,Friday the 13th: The Final Chapter,91 min,"Horror, Thriller",R,32.98,Joseph Zito,Erich Anderson,1984
2,Mercury Rising,111 min,"Action, Crime, Drama",R,32.94,Harold Becker,Bruce Willis,1998
3,Blade Runner,117 min,"Action, Drama, Sci-Fi",R,32.87,Ridley Scott,Harrison Ford,1982
4,The Last House on the Left,110 min,Thriller,R,32.75,Dennis Iliadis,Garret Dillahunt,2009
...,...,...,...,...,...,...,...,...
45,Angel Heart,113 min,"Horror, Mystery, Thriller",X,17.19,Alan Parker,Mickey Rourke,1987
46,Safe,94 min,"Action, Crime, Thriller",R,17.14,Boaz Yakin,Jason Statham,I 2012
47,Freejack,110 min,"Action, Crime, Sci-Fi",R,17.13,Geoff Murphy,Emilio Estevez,1992
48,Eastern Promises,100 min,"Crime, Drama, Thriller",R,17.11,David Cronenberg,Naomi Watts,2007


In [5]:
#master_df.to_csv('thrillers_751_1150rows.csv', index=False)